# Lecture 24:  NetworkX and PageRank Algorithm

__Reading material:__
- https://networkx.github.io/documentation/stable/tutorial.html#creating-a-graph<br>
  Read from "Creating a graph" to "Directed graphs"
- https://en.wikipedia.org/wiki/PageRank

__Optional:__
- https://www.geeksforgeeks.org/page-rank-algorithm-implementation/

We can use the equilibrium distribution of users on the internet as an approximation for the relative popularity of web pages. We start with an adjacency matrix, __A__, of which the elements represent links between pages. If a page has a number of links to other pages, we assume that a user picks one of them uniformly at random and moves to the next page. However, internet users do not navigate the web only by following links. (If you end up on a page with no links on it, you do not get stuck there for the rest of your life.) Users also occasionally move between pages with no links on them. We call this teleportation, and to simulate it in our model, we simply add a small constant to every element of __A__. This will translate to their being a non-zero probability of moving between any pair of pages, whether or not there is a link.


Here is the detail of the PageRank algorithm<br>
$$PR_0(x) = \frac{1}{N}$$

$$PR_i(x) = \frac{1-\alpha}{N}+\alpha\sum_{y\rightarrow x}\frac{PR_{i-1}(y)}{out(y)}$$

- $PR_i(x)$: the PageRank of node x at i-th iteration
- $\alpha$: a damping factor (we can set it to 0.85)
- $N$: the total number of nodes in the system
- $out(y)$: the number of outlinks of node y

### Graph visualization and PageRank in Networkx

In [1]:
import matplotlib.pyplot as plt 
import networkx as nx
from networkx.algorithms.approximation import min_weighted_dominating_set
G = nx.DiGraph()
# s = min_weighted_dominating_set(G, weight=None)
G.add_nodes_from(['A', 'B', 'C','D'])
G.add_edge('A', 'C')
G.add_edge('B', 'C')
G.add_edge('C', 'B')
G.add_edge('A', 'B')
G.add_edge('D', 'A')
G.add_edge('D', 'C')
nx.draw(G,with_labels=True)
plt.show()

AttributeError: module 'networkx' has no attribute 'path_graph'

In [ ]:
pr = nx.pagerank(G, alpha=0.85) # the default damping parameter alpha = 0.85
print pr

In [ ]:
# The following code is a simplified version of the function 
# which has been implemented in the networkx library.

def pagerank(G, alpha=0.85,
             max_iter=100, tol=1.0e-6, weight='weight'):    
    if len(G) == 0:
        return {}
 
    # Create a copy in (right) stochastic form
    W = nx.stochastic_graph(G, weight=weight)
    N = W.number_of_nodes()
 
    # Choose fixed starting vector
    x = dict.fromkeys(W, 1.0 / N)

    # Assign uniform personalization vector if not given
    p = dict.fromkeys(W, 1.0 / N)

    dangling_weights = p
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]
 
    # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x
        x = dict.fromkeys(xlast.keys(), 0)
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
        for n in x:
            # this matrix multiply looks odd because it is
            # doing a left multiply x^T=xlast^T*W
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]
 
        # check convergence, l1 norm
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N*tol:
            return x
    raise NetworkXError('pagerank: power iteration failed to converge '
                        'in %d iterations.' % max_iter)

In [ ]:
W = nx.stochastic_graph(G, weight='weight')
print W['A']
print W['B']
print W['C']
print W['D']